![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/misc/Multi_Model_Multi_Dataset.ipynb.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest

In [ ]:
!pip install "langtest[openai,transformers,evaluate]=2.3.0"

# Harness and Its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [4]:
# Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>


| Parameter  | Description |  
| - | - |
|**task**     |Task for which the model is to be evaluated (question-answering or summarization)|
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |

<br/>
<br/>

# OpenAI Model Testing For Question Answering

In this section, we dive into testing of OpenAI models in Question Answering task.

LangTest supports robustness tests for LLM testing for now.

In [5]:
yaml_content = """
prompt_config:
  "BoolQ":
    instructions: >
      You are an intelligent bot and it is your responsibility to make sure 
      to give a concise answer. Answer should be `true` or `false`.
    prompt_type: "instruct" # instruct for completion and chat for conversation(chat models)
    examples:
      - user:
          context: >
            The Good Fight -- A second 13-episode season premiered on March 4, 2018. 
            On May 2, 2018, the series was renewed for a third season.
          question: "is there a third series of the good fight?"
        ai:
          answer: "True"
      - user:
          context: >
            Lost in Space -- The fate of the castaways is never resolved, 
            as the series was unexpectedly canceled at the end of season 3.
          question: "did the robinsons ever get back to earth"
        ai:
          answer: "True"
  "NQ-open":
    instructions: >
      You are an intelligent bot and it is your responsibility to make sure 
      to give a short concise answer.
    prompt_type: "instruct" # completion
    examples:
      - user:
          question: "where does the electron come from in beta decay?"
        ai:
          answer: "an atomic nucleus"
      - user:
          question: "who wrote you're a grand ol flag?"
        ai:
          answer: "George M. Cohan"
  "MedQA":
    instructions: >
      You are an intelligent bot and it is your responsibility to make sure 
      to give a short concise answer.
    prompt_type: "instruct" # completion
    examples:
      - user:
          question: "what is the most common cause of acute pancreatitis?"
          options: "A. Alcohol\n B. Gallstones\n C. Trauma\n D. Infection"
        ai:
          answer: "B. Gallstones"
model_parameters:
    max_tokens: 64
tests:
    defaults:
        min_pass_rate: 0.65
    robustness:
        uppercase:
            min_pass_rate: 0.66
        dyslexia_word_swap:
            min_pass_rate: 0.6
        add_abbreviation:
            min_pass_rate: 0.6
        add_slangs:
            min_pass_rate: 0.6
        add_speech_to_text_typo:
            min_pass_rate: 0.6
"""

# with open("config.yaml", "w") as file:
#     file.write(yaml)
import yaml 

config = yaml.safe_load(yaml_content)

### Set environment for OpenAI

In [6]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"

### Multi Model, Multi Dataset Testing

In order to evaluate the multiple model's performance on multiple datasets, we can utilize a Jupyter notebook and provide a list of dictionaries to the `model` and `data` parameters. Each dictionary within the list should contain the following keys:

multiple models

```
model=[
    {"model": "gpt-3.5-turbo", "hub": "openai"},
    {"model": "gpt-4o", "hub": "openai"}],
],
```

multiple dataset

```
data=[
    {"data_source": "BoolQ", "split": "test-tiny"},
    {"data_source": "NQ-open", "split": "test-tiny"},
    {"data_source": "MedQA", "split": "test-tiny"},
    {"data_source": "LogiQA", "split": "test-tiny"},
],
```

Here, we specify different data sources and their corresponding splits for testing. This allows for a comprehensive evaluation of the model's performance across diverse datasets. The notebook can then be executed to assess how well the model generalizes to various types of questions and contexts presented in these datasets.

In [7]:
harness = Harness(
    task="question-answering",
    model=[
        {"model": "gpt-3.5-turbo", "hub": "openai"},
        {"model": "gpt-4o", "hub": "openai"}],
    data=[
        {"data_source": "BoolQ", "split": "test-tiny"},
        {"data_source": "NQ-open", "split": "test-tiny"},
        {"data_source": "MedQA", "split": "test-tiny"},
    ],
    config=config,
)

Test Configuration : 
 {
 "prompt_config": {
  "BoolQ": {
   "instructions": "You are an intelligent bot and it is your responsibility to make sure  to give a concise answer. Answer should be `true` or `false`.\n",
   "prompt_type": "instruct",
   "examples": [
    {
     "user": {
      "context": "The Good Fight -- A second 13-episode season premiered on March 4, 2018.  On May 2, 2018, the series was renewed for a third season.\n",
      "question": "is there a third series of the good fight?"
     },
     "ai": {
      "answer": "True"
     }
    },
    {
     "user": {
      "context": "Lost in Space -- The fate of the castaways is never resolved,  as the series was unexpectedly canceled at the end of season 3.\n",
      "question": "did the robinsons ever get back to earth"
     },
     "ai": {
      "answer": "True"
     }
    }
   ]
  },
  "NQ-open": {
   "instructions": "You are an intelligent bot and it is your responsibility to make sure  to give a short concise answer.\n",
 

## Robustness

For tests we used uppercase, Dyslexia Word Swap, Add Slangs, Insert Abbreviations and Speech to Text typos . Other available robustness tests for QA task are:
* `add_context`
* `add_contraction`
* `add_punctuation`
* `add_typo`
* `add_ocr_typo`
* `american_to_british`
* `british_to_american`
* `lowercase`
* `strip_punctuation`
* `titlecase`
* `uppercase`
* `number_to_word`
* `add_abbreviation`
* `add_speech_to_text_typo`
* `add_slangs`
* `dyslexia_word_swap`
* `multiple_perturbations`
* `adjective_synonym_swap`
* `adjective_antonym_swap`
* `strip_all_punctuation`

You can also set prompts and other model parameters in config. Possible parameters are:
* `user_promt:` Promt to be given to the model.
* `temperature:` Temperature of the model.
* `max_tokens:` Maximum number of output tokens allowed for model.

In [ ]:
harness.configure(
    {
        "model_parameters": {"max_tokens": 64},
        "tests": {
            "defaults": {"min_pass_rate": 0.65},
            "robustness": {
                "uppercase": {"min_pass_rate": 0.66},
                "dyslexia_word_swap": {"min_pass_rate": 0.60},
                "add_abbreviation": {"min_pass_rate": 0.60},
                "add_slangs": {"min_pass_rate": 0.60},
                "add_speech_to_text_typo": {"min_pass_rate": 0.60},
            },
        }
    }
)

➤ You can adjust the level of transformation in the sentence by using the "`prob`" parameter, which controls the proportion of words to be changed during robustness tests.

➤ **NOTE** : "`prob`" defaults to 1.0, which means all words will be transformed.
```
harness.configure(
{
 'tests': {
    'defaults': {'min_pass_rate': 0.65},
      'robustness': {
        'uppercase': {'min_pass_rate': 0.66, 'prob': 0.50},
        'dyslexia_word_swap':{'min_pass_rate': 0.60, 'prob': 0.70},
      }
  }
})

```

Here we have configured the harness to perform Five robustness tests and defined the minimum pass rate for each test.

In [8]:
#slice the data
harness.data = {k: v[:5] for k, v in harness.data.items()}

### Generating the test cases.

In [9]:
harness.generate()

                                     BoolQ                                      


Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


--------------------------------------------------------------------------------

                                    NQ-open                                     


Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


--------------------------------------------------------------------------------

                                     MedQA                                      


Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


--------------------------------------------------------------------------------



In [10]:
harness.testcases()

,dataset_name,model_name,category,test_type,original_context,original_question,perturbed_context,perturbed_question,options
0,BoolQ,gpt-3.5-turbo,robustness,uppercase,20 euro note -- Until now there has been only ...,is the first series 20 euro note still legal t...,20 EURO NOTE -- UNTIL NOW THERE HAS BEEN ONLY ...,IS THE FIRST SERIES 20 EURO NOTE STILL LEGAL T...,-
1,BoolQ,gpt-3.5-turbo,robustness,uppercase,2018–19 UEFA Champions League -- The final wil...,do the champions league winners get automatic ...,2018–19 UEFA CHAMPIONS LEAGUE -- THE FINAL WIL...,DO THE CHAMPIONS LEAGUE WINNERS GET AUTOMATIC ...,-
2,BoolQ,gpt-3.5-turbo,robustness,uppercase,Bullsnake -- Bullsnakes are very powerful cons...,can a bull snake kill a small dog,BULLSNAKE -- BULLSNAKES ARE VERY POWERFUL CONS...,CAN A BULL SNAKE KILL A SMALL DOG,-
3,BoolQ,gpt-3.5-turbo,robustness,uppercase,NBA playoffs -- All rounds are best-of-seven s...,are all nba playoff games best of 7,NBA PLAYOFFS -- ALL ROUNDS ARE BEST-OF-SEVEN S...,ARE ALL NBA PLAYOFF GAMES BEST OF 7,-
4,BoolQ,gpt-3.5-turbo,robustness,uppercase,Manchester station group -- The Manchester sta...,can i use my train ticket on the tram in manch...,MANCHESTER STATION GROUP -- THE MANCHESTER STA...,CAN I USE MY TRAIN TICKET ON THE TRAM IN MANCH...,-
...,...,...,...,...,...,...,...,...,...
127,MedQA,gpt-4o,robustness,add_speech_to_text_typo,-,A junior orthopaedic surgery resident is compl...,-,A junior orthopaedic surgery resident is compl...,A. Disclose the error to the patient but leave...
128,MedQA,gpt-4o,robustness,add_speech_to_text_typo,-,A 67-year-old man with transitional cell carci...,-,Ae 67-year-old man with transitional cell carc...,A. Inhibition of thymidine synthesis\nB. Inhib...
129,MedQA,gpt-4o,robustness,add_speech_to_text_typo,-,Two weeks after undergoing an emergency cardia...,-,Tue weeks after undergoing an emergency cardia...,A. Renal papillary necrosis\nB. Allergic inter...
130,MedQA,gpt-4o,robustness,add_speech_to_text_typo,-,A 39-year-old woman is brought to the emergenc...,-,A. 39-year-old woman is brought to the emergen...,"A. Coagulase-positive, gram-positive cocci for..."


harness.generate() method automatically generates the test cases (based on the provided configuration)

### Running the tests

In [11]:
harness.run()

                                     BoolQ                                      


Running testcases... : 100%|██████████| 22/22 [00:24<00:00,  1.13s/it]


--------------------------------------------------------------------------------

                                    NQ-open                                     


Running testcases... : 100%|██████████| 19/19 [00:25<00:00,  1.35s/it]


--------------------------------------------------------------------------------

                                     MedQA                                      


Running testcases... : 100%|██████████| 25/25 [00:32<00:00,  1.29s/it]

--------------------------------------------------------------------------------



Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

### Generated Results

In [12]:
harness.generated_results()

,dataset_name,model_name,dataset_name,category,test_type,original_context,original_question,perturbed_context,perturbed_question,options,expected_result,actual_result,pass
0,BoolQ,gpt-3.5-turbo,BoolQ,robustness,uppercase,20 euro note -- Until now there has been only ...,is the first series 20 euro note still legal t...,20 EURO NOTE -- UNTIL NOW THERE HAS BEEN ONLY ...,IS THE FIRST SERIES 20 EURO NOTE STILL LEGAL T...,-,False,True,False
1,BoolQ,gpt-3.5-turbo,BoolQ,robustness,uppercase,2018–19 UEFA Champions League -- The final wil...,do the champions league winners get automatic ...,2018–19 UEFA CHAMPIONS LEAGUE -- THE FINAL WIL...,DO THE CHAMPIONS LEAGUE WINNERS GET AUTOMATIC ...,-,True,True,True
2,BoolQ,gpt-3.5-turbo,BoolQ,robustness,uppercase,Bullsnake -- Bullsnakes are very powerful cons...,can a bull snake kill a small dog,BULLSNAKE -- BULLSNAKES ARE VERY POWERFUL CONS...,CAN A BULL SNAKE KILL A SMALL DOG,-,False,False,True
3,BoolQ,gpt-3.5-turbo,BoolQ,robustness,uppercase,NBA playoffs -- All rounds are best-of-seven s...,are all nba playoff games best of 7,NBA PLAYOFFS -- ALL ROUNDS ARE BEST-OF-SEVEN S...,ARE ALL NBA PLAYOFF GAMES BEST OF 7,-,True,True,True
4,BoolQ,gpt-3.5-turbo,BoolQ,robustness,uppercase,Manchester station group -- The Manchester sta...,can i use my train ticket on the tram in manch...,MANCHESTER STATION GROUP -- THE MANCHESTER STA...,CAN I USE MY TRAIN TICKET ON THE TRAM IN MANCH...,-,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,MedQA,gpt-4o,MedQA,robustness,add_speech_to_text_typo,-,A junior orthopaedic surgery resident is compl...,-,A junior orthopaedic surgery resident is compl...,A. Disclose the error to the patient but leave...,B. Disclose the error to the patient and put i...,B. Disclose the error to the patient and put i...,True
128,MedQA,gpt-4o,MedQA,robustness,add_speech_to_text_typo,-,A 67-year-old man with transitional cell carci...,-,Ae 67-year-old man with transitional cell carc...,A. Inhibition of thymidine synthesis\nB. Inhib...,A. Inhibition of thymidine synthesis,E. Cross-linking of DNA,False
129,MedQA,gpt-4o,MedQA,robustness,add_speech_to_text_typo,-,Two weeks after undergoing an emergency cardia...,-,Tue weeks after undergoing an emergency cardia...,A. Renal papillary necrosis\nB. Allergic inter...,C. Cholesterol embolization,C. Cholesterol embolization,True
130,MedQA,gpt-4o,MedQA,robustness,add_speech_to_text_typo,-,A 39-year-old woman is brought to the emergenc...,-,A. 39-year-old woman is brought to the emergen...,"A. Coagulase-positive, gram-positive cocci for...","B. Encapsulated, gram-negative coccobacilli fo...","B. Encapsulated, gram-negative coccobacilli fo...",True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

### Final Results

We can call `.report()` which summarizes the results giving information about pass and fail counts and overall test pass/fail flag.

In [13]:
harness.report()

Benchmarking Results  \
                                                                        fail_count   
dataset_name model_name    category   test_type                                      
BoolQ        gpt-3.5-turbo robustness uppercase                                  1   
                                      dyslexia_word_swap                         0   
                                      add_abbreviation                           1   
                                      add_slangs                                 0   
                                      add_speech_to_text_typo                    1   
             gpt-4o        robustness uppercase                                  1   
                                      dyslexia_word_swap                         1   
                                      add_abbreviation                           0   
                                      add_slangs                                 1   
                                      add_speech_to_text_typo                    0   
NQ-open      gpt-3.5-turbo robustness uppercase                                  2   
                                      dyslexia_word_swap                         2   
                                      add_abbreviation                           2   
                                      add_slangs                                 1   
                                      add_speech_to_text_typo                    2   
             gpt-4o        robustness uppercase                                  2   
                                      dyslexia_word_swap                         1   
                                      add_abbreviation                           2   
                                      add_slangs                                 1   
                                      add_speech_to_text_typo                    1   
MedQA        gpt-3.5-turbo robustness uppercase                                  1   
                                      dyslexia_word_swap                         0   
                                      add_abbreviation                           0   
                                      add_slangs                                 0   
                                      add_speech_to_text_typo                    0   
             gpt-4o        robustness uppercase                                  2   
                                      dyslexia_word_swap                         0   
                                      add_abbreviation                           1   
                                      add_slangs                                 0   
                                      add_speech_to_text_typo                    1   

                                                                          \
                                                              pass_count   
dataset_name model_name    category   test_type                            
BoolQ        gpt-3.5-turbo robustness uppercase                        4   
                                      dyslexia_word_swap               5   
                                      add_abbreviation                 4   
                                      add_slangs                       2   
                                      add_speech_to_text_typo          4   
             gpt-4o        robustness uppercase                        4   
                                      dyslexia_word_swap               4   
                                      add_abbreviation                 5   
                                      add_slangs                       1   
                                      add_speech_to_text_typo          5   
NQ-open      gpt-3.5-turbo robustness uppercase                        3   
                                      dyslexia_word_swap               2   
                                      add_abbreviation                 2   
       